In [ ]:
import os
from pprint import pprint
from google.cloud import storage

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'land-cover-mapping.json'

storage_client = storage.Client()

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()

In [ ]:
import glob
import shutil
import time
import json

import numpy as np
import tensorflow as tf
import folium

from osgeo import gdal
from osgeo import osr
import matplotlib.pyplot as plt

In [ ]:
# Change the UserName and ref_point_name
UserName = str(input("User Name: "))
ref_point_name = str(input('GEE Legacy Assets: '))

In [ ]:
cName = str(input('City Name: ')) 
lat_max = float(input('Maximum Latitute: ')) 
lat_min = float(input('Minimum Latitute: ')) 
lon_max = float(input('Maximum Longitude: '))
lon_min = float(input('Minimum Longitude: '))  
lyb = int(input('Begining Year of Landsat data: ')) # 1990
lye = int(input('Ending Year of Landsat data: ')) # 1990
ldb = int(input('Begining Day of Landsat data: ')) # 1
lde = int(input('Ending Day of Landsat data: ')) # 366
ep = int(input('Epoch: ')) # 25 100 200
sr = int(input('Spatial Resolution: ')) # 120
moas = int(input('Model output array of shape: ')) # 512, 128, 64
lr = float(input('Model learning rate: ')) # 0.001, 0.0001

In [ ]:
ts = time.time()
timestamp = int(ts)

blob_name = UserName + '_' + str(timestamp) + '/'

project_name = 'land-cover-mapping-340607'
bucket_name = 'landcovermapping'

gcs_client = storage.Client(project=project_name)
bucket = gcs_client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_string('', content_type='application/x-www-form-urlencoded;charset=UTF-8')

In [ ]:
def reduce_class_value(feat):
  return feat.set('class', ee.Number(feat.get('class')).subtract(1))


def cloudMask(img):
  return img.updateMask(img.select('BQA').lt(64))
  

def select_landsat(year):
  if (year >= 2013):
    return {
        'bands': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
        'FCCbands': ['B5', 'B4', 'B3'],
        'TextureBand': ['B5'],
        'bandsClassify': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7','B5_1'],
        'bandsGLCM': ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B5_asm', 'B5_contrast', "B5_corr", "B5_var", "B5_idm", "B5_savg", "B5_svar", "B5_sent", "B5_ent", "B5_dvar", "B5_dent", "B5_imcorr1", "B5_imcorr2","B5_maxcorr", "B5_diss",  "B5_inertia", "B5_shade", "B5_prom"],
        'Landsat': 'LANDSAT/LC08/C01/T1'
    }

  if (year >= 1999 and year <= 2002):
    return {
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
        'TextureBand': ['B4'],
        'bandsClassify': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','B4_1'],
        'bandsGLCM': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B4_asm', 'B4_contrast',"B4_corr", "B4_var","B4_idm","B4_savg", "B4_svar","B4_sent","B4_ent","B4_dvar","B4_dent","B4_imcorr1", "B4_imcorr2","B4_maxcorr","B4_diss","B4_inertia","B4_shade","B4_prom"],
        'FCCbands': ['B4', 'B3', 'B2'],
        'Landsat': 'LANDSAT/LE07/C01/T1'
    }

  if (year >= 1984 and year <= 1998 or year >=2003 and year <=2012):
    return{
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
        'bandsClassify': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','B4_1'],
        'bandsGLCM': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B4_asm', 'B4_contrast',"B4_corr", "B4_var","B4_idm","B4_savg", "B4_svar","B4_sent","B4_ent","B4_dvar","B4_dent","B4_imcorr1", "B4_imcorr2","B4_maxcorr","B4_diss","B4_inertia","B4_shade","B4_prom"],
        'FCCbands': ['B4', 'B3', 'B2'],
        'TextureBand': ['B4'],
        'Landsat': 'LANDSAT/LT05/C01/T1'
    }
  
  if (year == 1983):
    return{
        'bands': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],
        'bandsClassify': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7','B4_1'],
        'bandsGLCM': ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'B4_asm', 'B4_contrast',"B4_corr", "B4_var","B4_idm","B4_savg", "B4_svar","B4_sent","B4_ent","B4_dvar","B4_dent","B4_imcorr1", "B4_imcorr2","B4_maxcorr","B4_diss","B4_inertia","B4_shade","B4_prom"],
        'FCCbands': ['B4', 'B3', 'B2'],
        'TextureBand': ['B4'],
        'Landsat': 'LANDSAT/LT04/C01/T1'
    }
  
  if (year >= 1979 and year <= 1982):
    return {
        'bands': ['B4_median', 'B5_median', 'B6_median','B7_median','B6_median_1'],
        'bandsClassify': ['B4_median', 'B5_median', 'B6_median', 'B7_median'],
        'FCCbands': ['B6_median', 'B5_median', 'B4_median'],
        'TextureBand': ['B6_median'],
        'Landsat': 'LANDSAT/LM03/C01/T1'
    }

  if (year >= 1975 and year <= 1978):
    return {
        'bands': ['B4_median', 'B5_median', 'B6_median','B7_median','B6_median_1'],
        'bandsClassify': ['B4_median', 'B5_median', 'B6_median', 'B7_median'],
        'FCCbands': ['B6_median', 'B5_median', 'B4_median'],
        'TextureBand': ['B6_median'],
        'Landsat': 'LANDSAT/LM02/C01/T2'
    }

  if (year >= 1972 and year <= 1974):
    return {
        'bands': ['B4_median', 'B5_median', 'B6_median','B7_median','B6_median_1'],
        'bandsClassify': ['B4_median', 'B5_median', 'B6_median', 'B7_median'],
        'FCCbands': ['B6_median', 'B5_median', 'B4_median'],
        'TextureBand': ['B6_median'],
        'Landsat': 'LANDSAT/LM01/C01/T2'
    }

In [ ]:
def landsat_product(landsat_id):
  if(landsat_id == 'LANDSAT/LM01/C01/T2' or landsat_id == 'LANDSAT/LM02/C01/T2' or landsat_id == 'LANDSAT/LM03/C01/T1'):
    image = ee.ImageCollection(landsat_id).filterDate(str(gtYearBegin) + '-01-01',str(gtYearEnd) + '-12-31').filter(ROI[roi]['doyFilterLandsat']).filterBounds(out_ext).filterMetadata('CLOUD_COVER_LAND', 'less_than', 20).map(cloudMask).reduce(ee.Reducer.median())
    return {
      'region': image.geometry().bounds().getInfo(),
      'image': image,
      'image_int': image.toByte()       
    }
  else:
    image_col = ee.ImageCollection(landsat_id).filterDate(str(gtYearBegin) + '-01-01',str(gtYearEnd) + '-12-31').filter(ROI[roi]['doyFilterLandsat']).filterBounds(out_ext)
    return {
      'region': image_col.geometry().bounds().getInfo(),
      'image': ee.Algorithms.Landsat.simpleComposite(image_col, 50, cloud, 40, True),
      'image_int': ee.Algorithms.Landsat.simpleComposite(image_col, 50, cloud, 40, False)
  }

In [ ]:
def add_indices(params, product):
  
  if(params['Landsat']=='LANDSAT/LC08/C01/T1'):
    swir  = product['image'].select('B6')
    nir   = product['image'].select('B5')
    red   = product['image'].select('B4')
    green = product['image'].select('B3')

    ndvi  = nir.subtract(red).divide(nir.add(red))
    ndbi  = swir.subtract(nir).divide(swir.add(nir))
    bi    = ndbi.subtract(ndvi)
    ndwi  = green.subtract(swir).divide(green.add(swir))

    product['image'] = product['image'].addBands([ndvi, ndbi, bi, ndwi])
    product['image'] = product['image'].rename('B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11','ndvi','ndbi','bi','ndwi')

    params['bandsClassify'].extend(['ndvi', 'ndbi', 'bi', 'ndwi'])


  if(params['Landsat'] == 'LANDSAT/LE07/C01/T1'):
    swir  = product['image'].select('B5')
    nir   = product['image'].select('B4')
    red   = product['image'].select('B3')
    green = product['image'].select('B2')

    ndvi  = nir.subtract(red).divide(nir.add(red))
    ndbi  = swir.subtract(nir).divide(swir.add(nir))
    bi    = ndbi.subtract(ndvi)
    ndwi  = green.subtract(swir).divide(green.add(swir))

    product['image'] = product['image'].addBands([ndvi, ndbi, bi, ndwi])
    product['image'] = product['image'].rename('B1','B2','B3','B4','B5','B6_VCID_1','B6_VCID_2','B7','B8','ndvi','ndbi','bi','ndwi')

    params['bandsClassify'].extend(['ndvi', 'ndbi', 'bi', 'ndwi'])
    

  if(params['Landsat'] == 'LANDSAT/LM04/C01/T1' or params['Landsat'] == 'LANDSAT/LT05/C01/T1'):
    swir  = product['image'].select('B5')
    nir   = product['image'].select('B4')
    red   = product['image'].select('B3')
    green = product['image'].select('B2')

    ndvi  = nir.subtract(red).divide(nir.add(red))
    ndbi  = swir.subtract(nir).divide(swir.add(nir))
    bi    = ndbi.subtract(ndvi)
    ndwi  = green.subtract(swir).divide(green.add(swir))

    product['image'] = product['image'].addBands([ndvi, ndbi, bi, ndwi])
    product['image'] = product['image'].rename('B1','B2','B3','B4','B5','B6','B7','ndvi','ndbi','bi','ndwi')

    params['bandsClassify'].extend(['ndvi', 'ndbi', 'bi', 'ndwi'])


  if(params['Landsat'] == 'LANDSAT/LM01/C01/T2' or params['Landsat'] == 'LANDSAT/LM02/C01/T2' or params['Landsat'] == 'LANDSAT/LM03/C01/T1'):
    swir = product['image'].select('B6_median')
    nir = product['image'].select('B6_median')
    red = product['image'].select('B5_median')

    ndvi = nir.subtract(red).divide(nir.add(red))

    product['image'] = product['image'].addBands(ndvi)
    product['image'] = product['image'].rename('B4_median','B5_median','B6_median','B7_median','BQA_median','ndvi')

    params['bandsClassify'].append('ndvi')

In [ ]:
ROI = [
       {'cityName': cName, 'LatMax': lat_max, 'LatMin': lat_min, 'LonMax': lon_max, 'LonMin': lon_min, 'yearBegin':lyb, 'yearEnd':lye, 'doyFilter':ee.Filter.And(ee.Filter.greaterThanOrEquals('doy',  ldb), ee.Filter.lessThanOrEquals('doy',  lde)), 'doyFilterLandsat': ee.Filter.dayOfYear(ldb,lde)},
        # {'cityName': 'Ethiopia', 'LatMax': 14.911944087898716, 'LatMin': 11.9, 'LonMax': 39.102228429963795, 'LonMin': 36.663263586213795, 'yearBegin':1990, 'yearEnd':1990, 'doyFilter':ee.Filter.And(ee.Filter.greaterThanOrEquals('doy',  1), ee.Filter.lessThanOrEquals('doy',  366)), 'doyFilterLandsat': ee.Filter.dayOfYear(1,366)},
      #  {'cityName': 'Laguna Lake-Wet', 'LatMax': 14.5758304600000006, 'LatMin': 13.9641799899999999, 'LonMax': 121.6324386600000054, 'LonMin': 121.0053329499999961, 'yearBegin':1976, 'yearEnd':1976, 'doyFilter':ee.Filter.And(ee.Filter.greaterThanOrEquals('doy',  153), ee.Filter.lessThanOrEquals('doy',  305)), 'doyFilterLandsat': ee.Filter.dayOfYear(153, 305)},
      #  {'cityName': 'Laguna Lake-Dry', 'LatMax': 14.5758304600000006, 'LatMin': 13.9641799899999999, 'LonMax': 121.6324386600000054, 'LonMin': 121.0053329499999961, 'yearBegin':1976, 'yearEnd':1976, 'doyFilter':ee.Filter.Or(ee.Filter.greaterThanOrEquals('doy',  306), ee.Filter.lessThanOrEquals('doy',  152)), 'doyFilterLandsat': ee.Filter.Or(ee.Filter.dayOfYear(1, 152),ee.Filter.dayOfYear(306, 366))}
]

In [ ]:
roi= ''
kernel = ee.Kernel.gaussian(1)

out_ext = ''

cloud = 30
spatial_resolution = sr
n_sample = 10000
distance = 370000

base_distance = 5000
patch_size = 256
buffer_distance = base_distance * (round((spatial_resolution * 0.5 * patch_size)/base_distance) +1)

In [ ]:
ref_point = ee.FeatureCollection(ref_point_name)
ref_point = ref_point.map(reduce_class_value)

In [ ]:
for roi in range(0, len(ROI), 1):
  city_name = ROI[roi]['cityName']
  year_begin = ROI[roi]['yearBegin']
  year_end = ROI[roi]['yearEnd']

  USER_NAME = UserName
  IMAGE_FILE_PREFIX = city_name + '_' + str(year_begin) + '_' + str(year_end) + '_' + str(timestamp)
  FOLDER_NAME = USER_NAME + '_' + IMAGE_FILE_PREFIX + '_' + str(timestamp)

  for year in range(year_begin, year_end+1, 1):
    nSampleClass = []
    out_ext = ee.Geometry.Rectangle([ ROI[roi]['LonMin'], ROI[roi]['LatMin'], ROI[roi]['LonMax'], ROI[roi]['LatMax']])
    out_ext_center = out_ext.centroid()

    out_ext_buffer = out_ext.buffer(buffer_distance).bounds()

    center_lat = out_ext_center.getInfo()['coordinates'][1]
    center_lon = out_ext_center.getInfo()['coordinates'][0]

    landsat_params = select_landsat(year_begin)

    gtYearBegin = year_begin
    gtYearEnd = year_end


    product = landsat_product(landsat_params['Landsat'])
    
    LABEL_DATA = ref_point.filter(ee.Filter.And(ee.Filter.greaterThanOrEquals('year', gtYearBegin), ee.Filter.lessThanOrEquals('year', gtYearEnd), ROI[roi]['doyFilter']))
    LABEL_DATA = LABEL_DATA.filterBounds(product['region'])

    add_indices(landsat_params, product)

    gt_image = product['image']
    gt_image_int = product['image_int']
    gt_image_texture = gt_image.addBands(gt_image_int.select(landsat_params['TextureBand']).entropy(kernel))

    out_image = product['image'].clip(out_ext_buffer)
    out_image_int = product['image_int'].clip(out_ext_buffer)
    out_image_texture = out_image.addBands(out_image_int.select(landsat_params['TextureBand']).entropy(kernel))

In [ ]:
out_image

In [ ]:
mapid = out_image_texture.getMapId({'bands': [landsat_params['FCCbands'][0], landsat_params['FCCbands'][1], landsat_params['FCCbands'][2]], 'min': 0, 'max': 255})
map = folium.Map(location=[center_lat, center_lon]) 
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Out Image Texture',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [ ]:
BANDS = landsat_params['bandsClassify']
LABEL = 'class'
N_CLASSES = 4

FEATURE_NAMES = list(BANDS)
FEATURE_NAMES.append(LABEL)

In [ ]:
Bucket_Blob_Name = bucket_name + '/' + UserName + '_' + str(timestamp)

MY_BUCKET_PATH = os.getcwd()
FOLDER_PATH = MY_BUCKET_PATH + '/' + FOLDER_NAME
SHARED_DRIVE_PATH = '/content/drive/Shareddrives/MiyazakiLab/LC_mapping_models/'

TRAIN_FILE_PREFIX = 'Training_' + str(timestamp)
TEST_FILE_PREFIX = 'Testing_' + str(timestamp)

file_extension = '.tfrecord.gz'
TRAIN_FILE_PATH =  MY_BUCKET_PATH + '/' + FOLDER_NAME + '/' + 'sample/' + TRAIN_FILE_PREFIX + file_extension
TEST_FILE_PATH = MY_BUCKET_PATH + '/' + FOLDER_NAME + '/' + 'sample/' + TEST_FILE_PREFIX + file_extension

OUTPUT_IMAGE_TFR = MY_BUCKET_PATH + '/' + FOLDER_NAME + '/' + IMAGE_FILE_PREFIX+'.TFRecord'
OUTPUT_IMAGE_TIF = MY_BUCKET_PATH + '/' + FOLDER_NAME + '/' + IMAGE_FILE_PREFIX+'.tif'

EXPORT_REGION = out_ext
OUTPUT_ASSET_ID = 'users/' + USER_NAME + '/demo'

TRAIN_FILE_BUCKET = TRAIN_FILE_PREFIX + file_extension
TEST_FILE_BUCKET = TEST_FILE_PREFIX + file_extension

path = os.path.join(os.getcwd(), FOLDER_NAME)
os.mkdir(path)
# os.makedirs(dest_dir, exist_ok = True)

In [ ]:
sample = gt_image_texture.sampleRegions(
    collection = LABEL_DATA,
    properties = [LABEL],
    scale = spatial_resolution,
).randomColumn()

In [ ]:
training = sample.filter(ee.Filter.lt('random', 0.8))
testing = sample.filter(ee.Filter.gte('random', 0.8))

pprint({'training': training.first().getInfo()})
pprint({'testing': testing.first().getInfo()})

In [ ]:
training_task = ee.batch.Export.table.toCloudStorage(
  collection=training,
  description='Training Export',
  bucket=bucket_name,
  fileNamePrefix=TRAIN_FILE_PREFIX,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)
testing_task = ee.batch.Export.table.toCloudStorage(
  collection=testing,
  description='Testing Export',
  bucket=bucket_name,
  fileNamePrefix=TEST_FILE_PREFIX,
  fileFormat='TFRecord',
  selectors=FEATURE_NAMES)

In [ ]:
training_task.start()
testing_task.start()

In [ ]:
while training_task.active():
  print('Polling for task (id: {}).'.format(training_task.id))
  time.sleep(20)
print('Done with training export.')

while testing_task.active():
  print('Polling for task (id: {}).'.format(testing_task.id))
  time.sleep(20)
print('Done with testing export.')

In [ ]:
def move_blob(bucket_name, blob_name, destination_bucket_name, destination_blob_name):

    storage_client = storage.Client()

    source_bucket = storage_client.bucket(bucket_name)
    source_blob = source_bucket.blob(blob_name)
    destination_bucket = storage_client.bucket(destination_bucket_name)

    blob_copy = source_bucket.copy_blob(source_blob, destination_bucket, destination_blob_name)
    source_bucket.delete_blob(blob_name)


In [ ]:
move_blob(bucket_name, TRAIN_FILE_BUCKET, bucket_name, blob_name + TRAIN_FILE_BUCKET)
move_blob(bucket_name, TEST_FILE_BUCKET, bucket_name, blob_name + TEST_FILE_BUCKET)

In [ ]:
def download_file_from_bucket(blog_name, file_path, bucket_name):
    try:
        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(blog_name)
        with open(file_path, 'wb') as f:
            storage_client.download_blob_to_file(blob, f)
        print('Saved')
        return True
    except Exception as e:
        print(e)
        return False

In [ ]:
download_file_from_bucket(blob_name + TRAIN_FILE_BUCKET, os.path.join(os.getcwd(), FOLDER_NAME + '/'+ TRAIN_FILE_BUCKET), bucket_name)
download_file_from_bucket(blob_name + TEST_FILE_BUCKET, os.path.join(os.getcwd(), FOLDER_NAME + '/' + TEST_FILE_BUCKET), bucket_name)

In [ ]:
src_dir = MY_BUCKET_PATH + '/' + FOLDER_NAME
src_files = os.listdir(src_dir)

dest_dir = os.path.join(src_dir, 'sample')

os.makedirs(dest_dir, exist_ok = True)

for file in src_files:
    shutil.move(os.path.join(src_dir, file), dest_dir)

In [ ]:
print('Found training file.' if tf.io.gfile.exists(TRAIN_FILE_PATH) 
    else 'No training file found.')

print('Found testing file.' if tf.io.gfile.exists(TEST_FILE_PATH) 
    else 'No testing file found.')

In [ ]:
train_dataset = tf.data.TFRecordDataset(TRAIN_FILE_PATH, compression_type='GZIP')
test_dataset = tf.data.TFRecordDataset(TEST_FILE_PATH, compression_type='GZIP')

# print(iter(train_dataset).next())
# print(iter(test_dataset).next())

In [ ]:
columns = [tf.io.FixedLenFeature(shape=[1], dtype=tf.float32) for k in FEATURE_NAMES]

features_dict = dict(zip(FEATURE_NAMES, columns))

pprint(features_dict)

In [ ]:
def parse_tfrecord(example_proto):
  parsed_features = tf.io.parse_single_example(example_proto, features_dict)
  labels = parsed_features.pop(LABEL)
  return parsed_features, tf.cast(labels, tf.int32)

parsed_trainset = train_dataset.map(parse_tfrecord, num_parallel_calls=5)
parsed_testset = test_dataset.map(parse_tfrecord, num_parallel_calls=5)

# pprint(iter(parsed_trainset))
# pprint(iter(parsed_testset))

In [ ]:
batch_size = 100

def to_tuple(inputs, label):
  return (tf.transpose(list(inputs.values())), tf.one_hot(indices=label, depth=N_CLASSES))

input_dataset = parsed_trainset.map(to_tuple).batch(batch_size)
validate_dataset = parsed_testset.map(to_tuple).batch(batch_size)


In [ ]:
# model = tf.keras.models.Sequential([
#   tf.keras.layers.Dense(128, activation=tf.nn.relu),
#   tf.keras.layers.Dropout(0.1),
#   tf.keras.layers.Dense(64, activation=tf.nn.relu),
#   tf.keras.layers.Dropout(0.1),
#   tf.keras.layers.Dense(64, activation=tf.nn.relu),
#   tf.keras.layers.Dropout(0.1),
#   tf.keras.layers.Dense(64, activation=tf.nn.relu),
#   tf.keras.layers.Dense(64, activation=tf.nn.relu),
#   tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
# ])

#NEW NN STRUCTURE

drop = 0.05
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dropout(drop),
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dropout(drop),
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dropout(drop),
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dropout(drop),
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dropout(drop),
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dropout(drop),
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dropout(drop),
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dropout(drop),
  tf.keras.layers.Dense(moas, activation=tf.nn.relu),
  tf.keras.layers.Dense(N_CLASSES, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer= tf.keras.optimizers.Adam(
    learning_rate=lr),
    loss='categorical_crossentropy',
    metrics=['accuracy', 'mse'],
    # run_eagerly=True         
)

In [ ]:
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LearningRateScheduler

log_d = MY_BUCKET_PATH + '/' + FOLDER_NAME + '/' + 'model'
os.makedirs(log_d, exist_ok=True)
print(log_d)
def build_callbacks():
    checkpointer = ModelCheckpoint(filepath = log_d, verbose=0, save_best_only=True, monitor='val_loss')
    callbacks = [checkpointer]
    return callbacks
history = model.fit(x=input_dataset,validation_data=(validate_dataset), epochs=ep, callbacks=build_callbacks())

In [ ]:
import matplotlib.pyplot as plt
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
 model.evaluate(validate_dataset)

In [ ]:
image_export_options = {
    'patchDimensions': [patch_size, patch_size],
    'maxFileSize': 104857600,
    'compressed': True
}

image_task = ee.batch.Export.image.toCloudStorage(
  image=out_image_texture,
  description='Image Export',
  fileNamePrefix= IMAGE_FILE_PREFIX,
  bucket=bucket_name,
  scale=spatial_resolution,
  fileFormat='TFRecord',
  region=out_ext_buffer,
  formatOptions=image_export_options,
)

image_task.start()

In [ ]:
while image_task.active():
  print('Polling for task (id: {}).'.format(image_task.id))
  time.sleep(20)
print('Done with image export.')

In [ ]:
blobs = storage_client.list_blobs(bucket_name)

names= []

x = int(len(IMAGE_FILE_PREFIX))

for blob in blobs:
    if len(blob.name) > x:
        if blob.name[:x] == IMAGE_FILE_PREFIX:
            # print(blob.name)
            download_file_from_bucket(blob.name, FOLDER_PATH + '/'+ blob.name, bucket_name)
            move_blob(bucket_name, blob.name, bucket_name, blob_name + blob.name)
            # print(blob.name)


In [ ]:
tfrecord_list = glob.glob(MY_BUCKET_PATH +'/' + FOLDER_NAME + '/' + '*.tfrecord.gz')
tfrecord_list.sort()
pprint(tfrecord_list)

json_list = glob.glob(MY_BUCKET_PATH +'/' + FOLDER_NAME + '/' + '*.json')
pprint(json_list)

In [ ]:
with open(json_list[0], "r") as read_file:
   mixer = json.load(read_file)

mixer

In [ ]:
patch_width = mixer['patchDimensions'][0]
patch_height = mixer['patchDimensions'][1]
patches = mixer['totalPatches']
patch_dimensions_flat = [patch_width*patch_height, 1]

image_columns = [
                tf.io.FixedLenFeature(shape=patch_dimensions_flat, dtype=tf.float32) for k in BANDS
]

image_features_dict = dict(zip(BANDS, image_columns))

image_dataset = tf.data.TFRecordDataset(tfrecord_list, compression_type='GZIP')


In [ ]:
def parse_image(example_proto):
  return tf.io.parse_single_example(example_proto, image_features_dict)

In [ ]:
image_dataset = image_dataset.map(parse_image, num_parallel_calls=5)

image_dataset = image_dataset.flat_map(
    lambda features: tf.data.Dataset.from_tensor_slices(features)
)

image_dataset = image_dataset.map(
  lambda data_dict: (tf.transpose(list(data_dict.values())), )
)

image_dataset = image_dataset.batch(patch_width * patch_height)

In [ ]:
predictions = model.predict(image_dataset, steps=patches, verbose=1)
print(predictions[0])

In [ ]:
writer = tf.io.TFRecordWriter(OUTPUT_IMAGE_TFR)

patch = [[]]
cur_patch = 1
for prediction in predictions:
  patch[0].append(tf.argmax(prediction, 1))

  if (len(patch[0])==patch_width*patch_height):
    print('Done with Patch ' + str(cur_patch) + ' of ' + str(patches) + '...')
    example = tf.train.Example(
        features=tf.train.Features(
            feature={
                'prediction': tf.train.Feature(
                    float_list=tf.train.FloatList(
                        value=patch[0]
                    )
                )
            }
        )
    )

    writer.write(example.SerializeToString())
    patch=[[]]
    cur_patch +=1
  
writer.close()

In [ ]:
record_iterator = tf.compat.v1.python_io.tf_record_iterator(path=OUTPUT_IMAGE_TFR)

n_row = patches/mixer['patchesPerRow']
n_col = mixer['patchesPerRow']
current_row = 0
current_col = 0
counter = 0

for string_record in record_iterator:
  example = tf.train.Example()
  example.ParseFromString(string_record)
  values = np.array(example.features.feature['prediction'].float_list.value).reshape(patch_width, patch_height).astype(np.int8)

  if (current_col==0):
    horizontal_strip = values
  else:
    horizontal_strip = np.concatenate([horizontal_strip, values], axis=1)
  current_col += 1

  if (current_col == n_col):
    if (current_row==0):
      image = horizontal_strip
    else:
      image = np.concatenate([image, horizontal_strip], axis=0)
      horizontal_strip = []

    current_row +=1
    current_col = 0
    print(counter)
  counter +=1
print(counter)

In [ ]:
imgplot = plt.imshow(image, cmap=plt.get_cmap('jet'), vmin=0, vmax=1)
plt.show()

In [ ]:
affine = mixer['projection']['affine']['doubleMatrix']
geotransform = (affine[2], affine[0], affine[1], affine[5], affine[3], affine[4])
crs = int(mixer['projection']['crs'][-4:])
print(affine)
print(geotransform)
print(crs)

In [ ]:
ny, nx = image.shape
dst_ds = gdal.GetDriverByName('GTiff').Create(OUTPUT_IMAGE_TIF, nx, ny, 1, gdal.GDT_Byte)
dst_ds.SetGeoTransform(tuple(geotransform))
srs = osr.SpatialReference()
srs.ImportFromEPSG(crs)
dst_ds.SetProjection(srs.ExportToWkt())
dst_ds.GetRasterBand(1).WriteArray(image)
dst_ds.FlushCache()
dst_ds = None

print("Exorting " + OUTPUT_IMAGE_TIF + " completed.")

In [ ]:
fcc_image = out_image.select(landsat_params['FCCbands'])
fcc_prefix = city_name + '_' + str(year_begin) + '_' + str(year_end)+'_fcc'
fcc_task = ee.batch.Export.image.toCloudStorage(**{
    'image': fcc_image,
    'description': fcc_prefix,
    'bucket': bucket_name,
    'fileNamePrefix': fcc_prefix,
    'scale': spatial_resolution,
    'region': out_ext
})

fcc_task.start()

In [ ]:
while fcc_task.active():
  print('Polling for task (id: {}).'.format(fcc_task.id))
  time.sleep(20)
print('Done with fcc image export.')

In [ ]:
download_file_from_bucket(fcc_prefix + '.tif', FOLDER_PATH + '/'+ fcc_prefix + '.tif', bucket_name)
move_blob(bucket_name, fcc_prefix + '.tif' , bucket_name, blob_name + fcc_prefix + '.tif')

In [ ]:
lc_dir = MY_BUCKET_PATH + '/' + FOLDER_NAME + '/' + 'lc_map'
fcc_path = MY_BUCKET_PATH + '/' + FOLDER_NAME + '/' + fcc_prefix+'.tif'
ref_dir = MY_BUCKET_PATH + '/' + FOLDER_NAME + '/' + 'ref_map'

os.makedirs(lc_dir, exist_ok = True)
os.makedirs(ref_dir, exist_ok = True)

print(lc_dir)
# print(path)


for name in os.listdir(path):
    if name[:x] == IMAGE_FILE_PREFIX:
        # print (name[-12:])
        if name[-12:] == file_extension:
            print(name)
            shutil.move(os.path.join(path, name), lc_dir)
shutil.move(OUTPUT_IMAGE_TFR, lc_dir)    
shutil.move(OUTPUT_IMAGE_TIF, lc_dir)
shutil.move(fcc_path, ref_dir)